# 豆瓣爬虫分步骤演示

## 使用Python的requests库读取网页并提取返回的数据

下面演示如何获取到https://movie.douban.com/subject/11026735/ 的HTML代码

Tips：如何快速获取到对于的python代码？

![拷贝浏览器请求](resources/douban-1.png)
![转换成代码](resources/douban-2.png)

In [2]:
import requests

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://movie.douban.com/subject/11026735/comments?start=200&limit=20&status=P&sort=new_score',
    'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
}

response = requests.get('https://movie.douban.com/subject/11026735/', headers=headers)

显示前几百个字符

In [3]:
response.text[0:500]

'<!DOCTYPE html>\n<html lang="zh-CN" class="ua-mac ua-webkit">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n        超能陆战队 (豆瓣)\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <me'

## 通过正则表达式获取电影标题名称

正则表达式测试:https://regex101.com/r/AFRdGf/2

In [4]:
import re

regex = r"\"name\":\s\"(?P<name>.*?)\",\s*\"url\":\s\"(?P<url>.*?)\",\s*\"image\":\s\"(?P<image>.*?)\""
re.findall(regex,response.text)

[('超能陆战队 Big Hero 6',
  '/subject/11026735/',
  'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2614500883.webp')]

## 对于豆瓣，可以提取json+ld来获取相关信息

相关正则表达式：https://regex101.com/r/Ac3L5n/1

In [12]:
import json

regex = r"ld\+json\">(.*?)</script>"
jsonld = re.findall(regex,response.text, re.MULTILINE | re.DOTALL)[0]

movie = json.loads(jsonld)
movie['name']

'超能陆战队 Big Hero 6'

In [16]:
# 导演
[x['name'] for x in movie['author']]

['乔丹·罗伯茨 Jordan Roberts',
 '丹尼尔·吉尔森 Daniel Gerson',
 '罗伯特·L·贝尔德 Robert L. Baird',
 '唐·霍尔 Don Hall',
 '邓肯·鲁洛 Duncan Rouleau',
 '史提芬·T·西格尔 Steven T. Seagle',
 '乔·凯西 Joe Casey',
 '乔·凯利 Joe Kelly']

In [18]:
# 演员
[x['name'] for x in movie['actor']]

['斯科特·安第斯 Scott Adsit',
 '瑞恩·波特 Ryan Potter',
 '丹尼尔·海尼 Daniel Henney',
 'T·J·米勒 T.J. Miller',
 '杰米·钟 Jamie Chung',
 '小达蒙·韦恩斯 Damon Wayans Jr.',
 '珍尼希斯·罗德里格兹 Genesis Rodriguez',
 '詹姆斯·克伦威尔 James Cromwell',
 '艾伦·图代克 Alan Tudyk',
 '玛娅·鲁道夫 Maya Rudolph',
 '亚布拉哈姆·本鲁比 Abraham Benrubi',
 '凯蒂·洛斯 Katie Lowes',
 '比利·布什 Billy Bush',
 '丹尼尔·吉尔森 Daniel Gerson',
 '保罗·布里格斯 Paul Briggs',
 '夏洛特·古列齐 Charlotte Gulezian',
 'Dan Howell',
 '乔西·特立尼达 Josie Trinidad',
 '菅野美穗 Miho Kanno',
 '斯坦·李 Stan Lee',
 '大卫·肖内西 David Shaughnessy',
 '查尔斯·阿德勒 Charles Adler',
 '郝祥海 Xianghai Hao']

In [21]:
# 评分
float(movie['aggregateRating']['ratingValue'])

8.7